<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/caffe_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install caffe-cpu


In [ ]:
!wget https://github.com/shicai/MobileNet-Caffe/raw/master/mobilenet.caffemodel

In [ ]:
!wget https://raw.githubusercontent.com/shicai/MobileNet-Caffe/master/mobilenet_deploy.prototxt
!wget https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg

In [ ]:
import os
import os.path
import time
import sys
import ntpath
import caffe
caffe.set_mode_cpu()
from caffe.proto import caffe_pb2
import cv2
import numpy as np
import math
import string
from google.protobuf import text_format


model_path = "mobilenet_deploy.prototxt"
pretrained_path = "mobilenet.caffemodel"
input_name = "jet.jpeg"





def writeNPAryAsRaw(ipFrame, fileName, opDataType=np.float32, opScale=1):
    if opDataType != np.float32:
        qFrame = np.rint(ipFrame * opScale)
    else:
        qFrame = ipFrame
            
    fileHandle = open(fileName, 'wb')
    ip1DAry = np.reshape(qFrame, (1, np.prod(qFrame.shape)))
    ip1DAry = ip1DAry.astype(opDataType)
    fileHandle.write(ip1DAry)
    fileHandle.close()
       
def predict(model_path, pretrained_path, image, frameNum, blobs=None):
    net = caffe.Net(model_path, pretrained_path, caffe.TEST)
    #model = type('', (), {})()
    #model.net = net

    input_dims = net.blobs['data'].shape
    #output_dims = net.blobs['prob'].shape

    #label_margin = config.MARGIN
    print ("input dim from desc", input_dims[2], input_dims[3])
    #print ("output_dim from desc", output_dims[2], output_dims[3])

    batch_size, num_channels, input_height, input_width = input_dims
    caffe_in = np.zeros(input_dims, dtype=np.float32)

    caffe_in[0] = image.transpose([2, 0, 1])
    out_blobs = net.forward_all(blobs, **{net.inputs[0]: caffe_in})
    
    return out_blobs, net

def getLayerByName(net_proto, layer_name):
    for layer in net_proto.layer:
       if layer.name == layer_name:
          return layer
    return None
    

caffe.set_mode_cpu()
mean_pixel = [103.94,116.78,123.68]
num = 0
use_cur_scale = True 

net_proto = caffe_pb2.NetParameter()
text_format.Merge(open(model_path).read(), net_proto)

# moved image reading out from predict()
image = cv2.imread(input_name, 1);
image = cv2.resize(image, (224, 224))

image = image.astype(np.float32)- mean_pixel
image = image*0.017
blob_names =['prob']

out_blobs, net = predict(model_path, pretrained_path, image, num, blobs=blob_names)

dataOut = out_blobs['prob']
print(dataOut.shape)
argIndex = np.argsort(np.squeeze(dataOut))[::-1][:10]
print(argIndex)

input dim from desc 224 224
(1, 1000, 1, 1)
[403 895 744 657 913 833 812 908  51 755]
